# Get From PDF
This script implements the following steps:
* define the bounding box, which is represented through a list with the following shape: `[top,left,bottom,width]`. Data within the bounding box are expressed in cm. They must be converted to PDF points, since `tabula-py` requires them in this format. We set the conversion factor `fc = 28.28`.
* extract data using the `read_pdf()` function
* save data to a `pandas` dataframe.

In this example, we scan the pdf twice: firstly to extract the regions names, secondly, to extract tables. Thus we need to define two bounding boxes.

## Extract Regions names
Firstly, we define the bounding box to extract the regions:

In [247]:
box = [1.5, 22,3.8,26.741]
fc = 28.28
         
for i in range(0, len(box)):
    box[i] *= fc

Then, we import the `tabula-py` library and we define the list of pages from which we must extract information, as well as the file name.

In [248]:
import tabula as tb

pages = [3,5,6,8,9,10,12,14,16,18,22,24,26,28,30,32,34,36,38,40]
file = "source/Bolletino-sorveglianza-integrata-COVID-19_17-marzo-2020_appendix.pdf"

Now we can read the list of regions from the pdf. We use the `read_pdf()` function and we set the output format to `json`.

In [249]:
regions_raw = tb.read_pdf(file, pages=pages,area=[box],output_format="json")
regions_raw

[{'extraction_method': 'stream',
  'top': 42.42,
  'left': 622.16,
  'width': 134.07550048828125,
  'height': 65.04399871826172,
  'right': 756.2355,
  'bottom': 107.464,
  'data': [[{'top': 98.72,
     'left': 623.26,
     'width': 64.68998718261719,
     'height': 7.019999980926514,
     'text': 'Piemonte'}]]},
 {'extraction_method': 'stream',
  'top': 42.42,
  'left': 622.16,
  'width': 134.07550048828125,
  'height': 65.04399871826172,
  'right': 756.2355,
  'bottom': 107.464,
  'data': [[{'top': 70.64,
     'left': 623.26,
     'width': 87.00999450683594,
     'height': 7.019999980926514,
     'text': "Valle D'Aosta"}],
   [{'top': 94.7,
     'left': 624.78,
     'width': 127.40995025634766,
     'height': 5.519999980926514,
     'text': 'tori di riferimento regionale'}]]},
 {'extraction_method': 'stream',
  'top': 42.42,
  'left': 622.16,
  'width': 134.07550048828125,
  'height': 65.04399871826172,
  'right': 756.2355,
  'bottom': 107.464,
  'data': [[{'top': 70.64,
     'left':

We note that the produced output is very complex. However, the general structure contains the region name of the i-th region in the position `regions_raw[i]['data'][0][0]['text']`. We build a list with all the regions, by looping into the `region_raw` list.

In [250]:
regions = []
for i in range(0,len(regions_raw)):
    regions.append(regions_raw[i]['data'][0][0]['text'])

regions

['Piemonte',
 "Valle D'Aosta",
 'Lombardia',
 'Bolzano',
 'Trento',
 'Veneto',
 'FriuliVeneziaGiulia',
 'Liguria',
 'EmiliaRomagna',
 'Toscana',
 'Marche',
 'Lazio',
 'Abruzzo',
 'Molise',
 'Campania',
 'Puglia',
 'Basilicata',
 'Calabria',
 'Sicilia',
 'Sardegna']

## Extract the table for the first page (Piedmont region)

We define the bounding box and we multiply each value for the conversion factor `fc`. In order to understand how the mechanism works, firstly, we extract the table of the first page and then we generalize to all the pages. In our example, the first page corresponds to page 3.

In [251]:
box = [8,10,25,26]
for i in range(0, len(box)):
    box[i] *= fc  

Now we can read the pdf. In this case we set the `output_format` to `dataframe`. The result is stored in `tl`, which is a list. We can convert it to a dataframe, simply using `tl[0]`.

In [252]:
page = 3
tl = tb.read_pdf(file, pages=page,area=[box],output_format="dataframe", stream=True)
df = tl[0]
df.head()

,17 operatori sanitari,Unnamed: 1
0,NaN,NaN
1,NaN,NaN
2,Fascia d'età,Casi [n (%)]
3,0-9,4 (0.6%)
4,10-19,3 (0.5%)


We note that the columns names are wrong. In addition, the first three rows are wrong. For this reason, we can rename the columns names by using the dataframe function `rename()`.

In [253]:
df.rename(columns={ df.columns[0]: "Fascia d'età" , df.columns[1]: "Casi"}, inplace = True)
df.head()

,Fascia d'età,Casi
0,NaN,NaN
1,NaN,NaN
2,Fascia d'età,Casi [n (%)]
3,0-9,4 (0.6%)
4,10-19,3 (0.5%)


Now we can drop the first two rows by using the `dropna()` function.

In [254]:
df = df.dropna()
df.head()

,Fascia d'età,Casi
2,Fascia d'età,Casi [n (%)]
3,0-9,4 (0.6%)
4,10-19,3 (0.5%)
5,20-29,27 (4.2%)
6,30-39,39 (6%)


We can drop the new first row by selecting all the rows which do not contain this value.

In [255]:
df = df[df["Fascia d'età"] != "Fascia d'età"]
df.head(8)

,Fascia d'età,Casi
3,0-9,4 (0.6%)
4,10-19,3 (0.5%)
5,20-29,27 (4.2%)
6,30-39,39 (6%)
7,40-49,79 (12.2%)
8,50-59,103 (15.9%)
9,60-69,113 (17.5%)
10,70-79,137 (21.2%)


Now we add a new column to `df`, called `Regione` which contains the region name. We scan the `pages` list to extract the index of the current region.

In [256]:
region_column = []
for i in range(0, len(df)):
    index = pages.index(page)
    region_column.append(regions[index])

df['Regione'] = region_column
df.head()

,Fascia d'età,Casi,Regione
3,0-9,4 (0.6%),Piemonte
4,10-19,3 (0.5%),Piemonte
5,20-29,27 (4.2%),Piemonte
6,30-39,39 (6%),Piemonte
7,40-49,79 (12.2%),Piemonte


## Extract all pages
Now we can generalize the previous code to extract the tables of all the pages. Firstly, we build an empty `DataFrame`, which will contain the values for all the regions. We will use the `pd.concat()` function to concatenate all the tables of alle the pages.
We scan all the pages contained in the `pages` list.

In [233]:
import pandas as pd
df = pd.DataFrame()
for page in pages:
    
    index = pages.index(page)
    region = regions[index]
    print(region)
    
    tl = tb.read_pdf(file, pages=page,area=[box],output_format="dataframe", stream=True)
    
    dft = tl[0]
    dft.rename(columns={ dft.columns[0]: "Fascia d'età" , dft.columns[1]: "Casi"}, inplace = True)
    
    region_column = []
    for i in range(0, len(dft)):
        region_column.append(region)
    dft['Regione'] = region_column
    
    df = pd.concat([df, dft])

Piemonte
Valle D'Aosta
Lombardia
Bolzano
Trento
Veneto
FriuliVeneziaGiulia
Liguria
EmiliaRomagna
Toscana
Marche
Lazio
Abruzzo
Molise
Campania
Puglia
Basilicata
Calabria
Sicilia
Sardegna


As in the previous case, we drop all wrong records.

In [234]:
df.dropna(inplace=True)
df = df[df["Fascia d'età"] != "Fascia d'età"]
df.head(30)

,Fascia d'età,Casi,Regione
3,0-9,4 (0.6%),Piemonte
4,10-19,3 (0.5%),Piemonte
5,20-29,27 (4.2%),Piemonte
6,30-39,39 (6%),Piemonte
7,40-49,79 (12.2%),Piemonte
8,50-59,103 (15.9%),Piemonte
9,60-69,113 (17.5%),Piemonte
10,70-79,137 (21.2%),Piemonte
11,80-89,108 (16.7%),Piemonte
12,&gt;90,28 (4.3%),Piemonte


## Save results to CSV
Now we can save the result as a csv file.

In [235]:
df.to_csv('output.csv')